In [11]:
from sc2 import maps
from sc2.player import Bot, Computer
from sc2.main import run_game
from sc2.data import Race, Difficulty
from sc2.bot_ai import BotAI
from sc2.ids.unit_typeid import UnitTypeId
import random
import time
import pickle
import numpy as np
import math
import cv2

In [12]:
# from sc2.game_state import GameState


In [13]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import nest_asyncio
nest_asyncio.apply()

In [14]:
class WorkerRushBot(BotAI):
    async def on_step(self, iteration: int):        
        while True:
            try:
                with open('transaction.pkl','rb') as f:
                    transaction = pickle.load(f)
                if transaction['action'] is not None:
                    break
                time.sleep(0.1)
            except Exception as e:
                time.sleep(0.1)
                pass
        
        action = transaction['action']
        print(f'迭代次数：{iteration}')
        await self.distribute_workers()
        
        #0:扩中枢、训练探机、吸收间、水晶塔
        if action == 0:
            print(f'action={action}')
            try:
                have_builded = False
                if self.supply_left < 4:
                    if self.can_afford(UnitTypeId.PYLON) and self.already_pending(UnitTypeId.PYLON)==0:
                            await self.build(UnitTypeId.PYLON,near=random.choice(self.townhalls))
                            have_builded = True
                            print('建设水晶塔')

                if not have_builded:
                    for nexus in self.townhalls:
                        #探机
                        workers_count = len(self.workers.closer_than(10,nexus))
                        if workers_count < 22:
                            if self.can_afford(UnitTypeId.PROBE) and nexus.is_idle:
                                nexus.train(UnitTypeId.PROBE)
                                have_builded = True
                                print('训练探机')
                        #吸收间
                        for vespene in self.vespene_geyser.closer_than(10,nexus):
                            if self.can_afford(UnitTypeId.ASSIMILATOR) and not self.structures(UnitTypeId.ASSIMILATOR).closer_than(2,vespene).exists:
                                await self.build(UnitTypeId.ASSIMILATOR,vespene)  
                                have_builded = True
                                print('建设吸收间')

                if not have_builded:
                    if self.can_afford(UnitTypeId.NEXUS) and self.already_pending(UnitTypeId.NEXUS)==0:
                        await self.expand_now()
                        print('扩建中枢')
            except Exception as e:
                print(e)
            
        #1:传送门、控制核心、星门
        elif action == 1:
            print(f'action={action}')
            try:
                for nexus in self.townhalls:
                    #传送门
                    if not self.structures(UnitTypeId.GATEWAY).closer_than(10,nexus).exists:
                        if self.can_afford(UnitTypeId.GATEWAY) and self.already_pending(UnitTypeId.GATEWAY)==0:
                            await self.build(UnitTypeId.GATEWAY,near=nexus)
                            print('建设传送门')

                    #控制核心
                    if not self.structures(UnitTypeId.CYBERNETICSCORE).closer_than(10,nexus).exists:
                        if self.can_afford(UnitTypeId.CYBERNETICSCORE) and self.already_pending(UnitTypeId.CYBERNETICSCORE)==0:
                            await self.build(UnitTypeId.CYBERNETICSCORE,near=nexus)
                            print('建设控制核心')

                    #星门
                    if not self.structures(UnitTypeId.STARGATE).closer_than(10,nexus).exists:
                        if self.can_afford(UnitTypeId.STARGATE) and self.already_pending(UnitTypeId.STARGATE)==0:
                            await self.build(UnitTypeId.STARGATE,near=nexus)
                            print('建设星门')
            except Exception as e:
                print(e)
        
        #2:虚空辉光舰
        elif action == 2:
            print(f'action={action}')
            try:
                for sg in self.structures(UnitTypeId.STARGATE).ready.idle:
                    if self.can_afford(UnitTypeId.VOIDRAY):
                        sg.train(UnitTypeId.VOIDRAY)
                        print('训练虚空辉光舰')
            except Exception as e:
                print(e)
        
        #3:侦查
        elif action == 3:
            print(f'action={action}')
            try:
                self.last_sent
            except:
                self.last_sent = 0
            if (iteration - self.last_sent) > 200:
                try:
                    if self.units(UnitTypeId.PROBE).idle.exists:
                        probe = random.choice(self.units(UnitTypeId.PROBE).idle)
                    else:
                        probe = random.choice(self.units(UnitTypeId.PROBE))
                    probe.attack(self.enemy_start_locations[0])
                    self.last_sent = iteration
                    print('侦查')
                except:
                    pass
        
        #4:进攻
        elif action == 4:
            print(f'action={action}')
            try:
                for voidray in self.units(UnitTypeId.VOIDRAY).idle:
                    if self.enemy_units.closer_than(10,voidray):
                        voidray.attack(random.choice(self.enemy_units.closer_than(10,voidray)))
                    elif self.enemy_structures.closer_than(10,voidray):
                        voidray.attack(random.choice(self.enemy_structures.closer_than(10,voidray)))
                    elif self.enemy_units:
                        voidray.attack(random.choice(self.enemy_units))
                    elif self.enemy_structures:
                        voidray.attack(random.choice(self.enemy_structures))
                    elif self.enemy_start_locations:
                        voidray.attack(self.enemy_start_locations[0])
                    print('星空辉光舰进攻')
            except Exception as e:
                print(e)
        
        #5:撤退
        elif action == 5:
            print(f'action={action}')
            try:
                if self.units(UnitTypeId.VOIDRAY).amount > 0:
                    for voidray in self.units(UnitTypeId.VOIDRAY):
                        voidray.attack(self.start_location)
                    print('撤退')
            except Exception as e:
                print(e)            
        
        #画图
        map = np.zeros((self.game_info.map_size[0],self.game_info.map_size[1],3),dtype = np.uint8)
        
        #1:绘制矿产资源
        for mineral in self.mineral_field:
            pos = mineral.position
            c = [175,255,255]
            fraction = mineral.mineral_contents/1800
            if mineral.is_visible:
                map[math.ceil(pos.y)][math.ceil(pos.x)] = [int(fraction*i) for i in c]
            else:
                map[math.ceil(pos.y)][math.ceil(pos.x)] = [50,50,50]
        
        #2:绘制瓦斯资源
        for vespene in self.vespene_geyser:
            pos = vespene.position
            c = [255,175,255]
            fraction = vespene.vespene_contents/2250
            if vespene.is_visible:
                map[math.ceil(pos.y)][math.ceil(pos.x)] = [int(fraction*i) for i in c]
            else:
                map[math.ceil(pos.y)][math.ceil(pos.x)] = [50,50,50]
                
        #3:绘制基础设施
        for structure in self.structures:
            pos = structure.position
            if structure.type_id == UnitTypeId.NEXUS:
                c = [255,255,175]
            else:
                c = [0,255,175]
            fraction = structure.health_percentage
            map[math.ceil(pos.y)][math.ceil(pos.x)] = [int(fraction*i) for i in c]
            
        #4:绘制武器装备
        for unit in self.units:
            pos = unit.position
            if unit.type_id == UnitTypeId.VOIDRAY:
                c = [255,0,0]
            else:
                c = [175,255,0]
            fraction = unit.health_percentage
            map[math.ceil(pos.y)][math.ceil(pos.x)] = [int(fraction*i) for i in c]
            
        #5:绘制敌人的起始位置画面
        for enermy_location in self.enemy_start_locations:
            pos = enermy_location.position
            c = [0,0,255]
            map[math.ceil(pos.y)][math.ceil(pos.x)] = c
            
        #6:绘制敌人的基础设施
        for structure in self.enemy_structures:
            pos = structure.position
            c = [0,100,255]
            fraction = structure.health_percentage
            map[math.ceil(pos.y)][math.ceil(pos.x)] = [int(fraction*i) for i in c]
            
        #7:绘制敌人的武器装备
        for unit in self.enemy_units:
            pos = unit.position
            c = [100,0,255]
            fraction = unit.health_percentage
            map[math.ceil(pos.y)][math.ceil(pos.x)] = [int(fraction*i) for i in c]
            
        #计算奖励值
        reward = 0
        try:
            for voidray in self.units(UnitTypeId.VOIDRAY):
                if voidray.is_attacking and voidray.target_in_range:
                    if self.enemy_structures.closer_than(8,voidray) or self.enemy_units.closer_than(8,voidray):
                        reward += 0.015
        except Exception as e:
            print(f'reward error:{e}')
            reward = 0
        
        if iteration%10 == 0:
            print(f'iteration:{iteration},RW:{reward},VR:{self.units(UnitTypeId.VOIDRAY).amount}')
            
  
        cv2.imshow('map',cv2.flip(cv2.resize(map,None,fx=4,fy=4,interpolation=cv2.INTER_NEAREST),0))
        cv2.waitKey(1)
        
        
        transaction = {'observation':map,'reward':reward,'action':None,'done':False}
        with open('transaction.pkl','wb') as f:
            pickle.dump(transaction,f)

In [ ]:
result = run_game(maps.get('AscensiontoAiurLE'), [
    Bot(Race.Protoss, WorkerRushBot()),
    Computer(Race.Zerg, Difficulty.Hard)
], realtime=True)

2022-09-13 08:30:45.707 | INFO     | sc2.protocol:_execute:72 - Client status changed to Status.launched (was None)
2022-09-13 08:30:45.708 | INFO     | sc2.controller:create_game:37 - Creating new game
2022-09-13 08:30:45.709 | INFO     | sc2.controller:create_game:38 - Map:     AscensiontoAiurLE
2022-09-13 08:30:45.711 | INFO     | sc2.controller:create_game:39 - Players: Bot WorkerRushBot(Protoss), Computer Hard(Zerg, RandomBuild)
2022-09-13 08:30:45.715 | INFO     | sc2.protocol:_execute:72 - Client status changed to Status.init_game (was Status.launched)
2022-09-13 08:30:52.074 | INFO     | sc2.protocol:_execute:72 - Client status changed to Status.in_game (was None)
2022-09-13 08:30:52.075 | INFO     | sc2.main:_play_game:221 - Player 1 - Bot WorkerRushBot(Protoss)
2022-09-13 08:32:38.284 | INFO     | sc2.sc2process:kill_all:33 - kill_switch: Process cleanup for 1 processes
2022-09-13 08:32:42.136 | INFO     | sc2.sc2process:kill_all:33 - kill_switch: Process cleanup for 1 proces

In [ ]:
with open('result.txt','a') as f:
    f.write(f'{result}\n')
    
if str(result) == 'Result.Victory':
    print('Victory!')
    rwd = 500
else:
    rwd = -500
    
map = np.zeros((224,224,3),dtype = np.uint8)
transaction = {'observation':map,'reward':rwd,'action':None,'done':True}
with open('transaction.pkl','wb') as f:
    pickle.dump(transaction,f)
    
cv2.destroyAllWindows()
cv2.waitKey(1)
time.sleep(1)